In [15]:
import os
import sys
import json
import numpy as np
import re
import copy
from scipy import ndimage

In [2]:
def read_ARC_JSON(filepath):
    """Given a filepath, read in the ARC task data which is in JSON
    format. Extract the train/test input/output pairs of
    grids. Convert each grid to np.array and return train_input,
    train_output, test_input, test_output."""

    # Open the JSON file and load it
    data = json.load(open(filepath))

    # Extract the train/test input/output grids. Each grid will be a
    # list of lists of ints. We convert to Numpy.
    train_input = [np.array(data['train'][i]['input'])
                   for i in range(len(data['train']))]
    train_output = [np.array(data['train'][i]['output'])
                    for i in range(len(data['train']))]
    test_input = [np.array(data['test'][i]['input'])
                  for i in range(len(data['test']))]
    test_output = [np.array(data['test'][i]['output'])
                   for i in range(len(data['test']))]

    return (train_input, train_output, test_input, test_output)

In [3]:
train_input, train_output, test_input, test_output = read_ARC_JSON("../data/training/dc1df850.json")

In [4]:
print(type(train_input))

<class 'list'>


In [5]:
def get_closed_area(arr):
    # depth first search
    green = 3
    H, W = arr.shape
    Dy = [0, -1, 0, 1]
    Dx = [1, 0, -1, 0]
    arr_padded = np.pad(arr, ((1, 1), (1, 1)), "constant", constant_values=0)
    searched = np.zeros(arr_padded.shape, dtype=bool)
    searched[0, 0] = True
    q = [(0, 0)]
    while q:
        y, x = q.pop()
        for dy, dx in zip(Dy, Dx):
            y_, x_ = y+dy, x+dx
            if not 0 <= y_ < H+2 or not 0 <= x_ < W+2:
                continue
            if not searched[y_][x_] and arr_padded[y_][x_] == 0:
                q.append((y_, x_))
                searched[y_, x_] = True
    res = searched[1:-1, 1:-1]
    res |= arr == green
    return ~res

In [22]:
def solve_dc1df850(x):
    x1 = copy.deepcopy(x)
    red, blue = 2, 1

    struct2 = ndimage.generate_binary_structure(2, 2)
    final_arr = ndimage.binary_dilation(x, structure=struct2, mask=2).astype(x.dtype)
    return final_arr

In [23]:
for x, y in zip(train_input, train_output):
        # yhat = solve(x)
        print("X:")
        print(x)
        print("Solved:")
        print(solve_dc1df850(x))
        # show_result(x, y, yhat)


X:
[[2 0 0 0 0]
 [0 0 0 2 0]
 [0 0 0 0 0]
 [0 6 0 0 0]
 [0 0 0 0 0]]
Solved:


RuntimeError: mask and input must have equal sizes